In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
df = pd.read_excel("/content/Input.xlsx")

In [100]:
df.head()

,URL_ID,URL
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...
1,38.0,https://insights.blackcoffer.com/what-if-the-c...
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...
3,40.0,https://insights.blackcoffer.com/will-machine-...
4,41.0,https://insights.blackcoffer.com/will-ai-repla...


In [ ]:
df.iloc[0]

URL_ID                                                 37.0
URL       https://insights.blackcoffer.com/ai-in-healthc...
Name: 0, dtype: object

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}

URL = "https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/"
source = requests.get(URL, headers=headers).text
data = BeautifulSoup(source, 'lxml')

In [ ]:
title = data.find("h1",class_="entry-title").text
title

'AI in healthcare to Improve Patient Outcomes'

In [ ]:
desc = data.find("div",class_="td-post-content").text

In [ ]:
desc

'\nIntroduction\n“If anything kills over 10 million people in the next few decades, it will be a highly infectious virus rather than a war. Not missiles but microbes.” Bill Gates’s remarks at a TED conference in 2014, right after the world had avoided the Ebola outbreak. When the new, unprecedented, invisible virus hit us, it met an overwhelmed and unprepared healthcare system and oblivious population. This public health emergency demonstrated our lack of scientific consideration and underlined the alarming need for robust innovations in our health and medical facilities. For the past few years, artificial intelligence has proven to be of tangible potential in the healthcare sectors, clinical practices, translational medical and biomedical research.\nAfter the first case was detected in China on December 31st 2019, it was an AI program developed by BlueDot that alerted the world about the pandemic. It was quick to realise AI’s ability to analyse large chunks of data could help in detec

In [ ]:
import string
import re
import nltk
import os
punch = string.punctuation

In [ ]:
 nltk.download('stopwords')
 nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def clean_text(text):
    non_punch = ''
    for i in text:
        if i not in string.punctuation:
            non_punch += i
    split_text = re.split('\W+',non_punch)
    stop = nltk.corpus.stopwords.words('english')
    wn = nltk.WordNetLemmatizer()
    return " ".join([wn.lemmatize(word).lower() for word in split_text])

In [ ]:
path = "/content/stopwords"
dir_list = os.listdir(path)

In [ ]:
dir_list

['StopWords_Generic.txt',
 'StopWords_Auditor.txt',
 'StopWords_Geographic.txt',
 'StopWords_Names.txt',
 'StopWords_GenericLong.txt',
 'StopWords_Currencies.txt',
 'StopWords_DatesandNumbers.txt']

In [ ]:
stopWords = []
for p in dir_list:
  stopWords += open(path+"/"+p,"r", encoding='utf-8').read().split("\n")

In [ ]:
len(stopWords)

14111

In [101]:
all_text = title+" "+desc
content = clean_text(all_text)

In [102]:
content

'ai in healthcare to improve patient outcomes introduction if anything kill over 10 million people in the next few decade it will be a highly infectious virus rather than a war not missile but microbe bill gates s remark at a ted conference in 2014 right after the world had avoided the ebola outbreak when the new unprecedented invisible virus hit u it met an overwhelmed and unprepared healthcare system and oblivious population this public health emergency demonstrated our lack of scientific consideration and underlined the alarming need for robust innovation in our health and medical facility for the past few year artificial intelligence ha proven to be of tangible potential in the healthcare sector clinical practice translational medical and biomedical research after the first case wa detected in china on december 31st 2019 it wa an ai program developed by bluedot that alerted the world about the pandemic it wa quick to realise ai s ability to analyse large chunk of data could help in

In [103]:
def newContent(text):
  newcontent = []
  for item in text.split(" "):
    if item.upper() not in stopWords:
      newcontent.append(item)
  return newcontent

In [104]:
newc = newContent(content)

## Positive and Negative words

In [105]:
pword = open("/content/master/positive-words.txt","r").read().split("\n")

In [106]:
nword = open("/content/master/negative-words.txt","r", encoding='utf-8').read().split("\n")

In [107]:
positive_score = 0
negative_score = 0
for word in newc:
  if word.lower() in pword:
    positive_score += 1
  if word.lower() in nword:
    negative_score += 1

polarity_score = (positive_score - negative_score)/ ((positive_score + negative_score) + 0.000001)

In [108]:
subjectivity_score = (positive_score + negative_score)/ (len(newc) + 0.000001)

In [109]:
subjectivity_score

0.10940325487395726

## Sentance

In [111]:
sentences= nltk.sent_tokenize(" ".join(all_text))
len(sentences)

9

In [112]:
def count_syllables(word):
    word = word.lower()
    counter = 0
    is_previous_vowel = False
    for index, value in enumerate(word):
        if value in ["a", "e", "i", "o", "u", "y"]:
            if index == len(word) - 1:
                if value == "e":
                    if counter == 0:
                        counter += 1
                else:
                    counter += 1
            else:
                if is_previous_vowel == True:
                    counter += 1
                    is_previous_vowel = False
                    break
            is_previous_vowel = True
        else:
            if is_previous_vowel == True:
                counter += 1
            is_previous_vowel = False
    return counter
count_syllables("coding")

2

In [113]:
percentage_of_complex_words = 0
for word in newc:
  if count_syllables(word) > 2:
    percentage_of_complex_words += 1
percentage_of_complex_words

386

In [137]:
sents = all_text.split('.')
avg_sent_len = sum(len(x.split()) for x in sents) / len(sents)

In [138]:
avg_sent_len

23.064102564102566

In [114]:
syllables_per_word = percentage_of_complex_words/len(newc)
syllables_per_word

0.34900542495479203

In [115]:
average_sentence_length = len(newc)/len(sentences)
average_sentence_length

122.88888888888889

In [116]:
fog_index = 0.4 * (average_sentence_length + percentage_of_complex_words)
fog_index

203.55555555555557

In [121]:
pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
pronouns = pronounRegex.findall(all_text)
len(pronouns)

1

In [131]:
avg_word_len = (len(content)-len(newc))/len(newc)
avg_word_len

9.616636528028932

In [135]:
output = {
    "URL_ID":[],
    "URL":[],
    "POSITIVE SCORE":[],
    "NEGATIVE SCORE":[],
    "POLARITY SCORE":[],
    "SUBJECTIVITY SCORE":[],
    "AVG SENTENCE LENGTH":[],
    "PERCENTAGE OF COMPLEX WORDS":[],
    "FOG INDEX":[],
    "AVG NUMBER OF WORDS PER SENTENCE":[],
    "COMPLEX WORD COUNT":[],
    "WORD COUNT":[],
    "SYLLABLE PER WORD":[],
    "PERSONAL PRONOUNS":[],
    "AVG WORD LENGTH":[],
}

In [175]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import string
import re
import nltk
import os

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

punch = string.punctuation

df = pd.read_excel("/content/Input.xlsx")
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}

def clean_text(text):
    non_punch = ''
    for i in text:
        if i not in string.punctuation:
            non_punch += i
    split_text = re.split('\W+',non_punch)
    stop = nltk.corpus.stopwords.words('english')
    wn = nltk.WordNetLemmatizer()
    return " ".join([wn.lemmatize(word).lower() for word in split_text])

def newContent(text):
  newcontent = []
  for item in text.split(" "):
    if item.upper() not in stopWords:
      newcontent.append(item)
  return newcontent

def count_syllables(word):
    word = word.lower()
    counter = 0
    is_previous_vowel = False
    for index, value in enumerate(word):
        if value in ["a", "e", "i", "o", "u", "y"]:
            if index == len(word) - 1:
                if value == "e":
                    if counter == 0:
                        counter += 1
                else:
                    counter += 1
            else:
                if is_previous_vowel == True:
                    counter += 1
                    is_previous_vowel = False
                    break
            is_previous_vowel = True
        else:
            if is_previous_vowel == True:
                counter += 1
            is_previous_vowel = False
    return counter


path = "/content/stopwords"
dir_list = os.listdir(path)
stopWords = []
for p in dir_list:
  stopWords += open(path+"/"+p,"r", encoding='utf-8').read().split("\n")

output = {
    "URL_ID":[],
    "URL":[],
    "POSITIVE SCORE":[],
    "NEGATIVE SCORE":[],
    "POLARITY SCORE":[],
    "SUBJECTIVITY SCORE":[],
    "AVG SENTENCE LENGTH":[],
    "PERCENTAGE OF COMPLEX WORDS":[],
    "FOG INDEX":[],
    "AVG NUMBER OF WORDS PER SENTENCE":[],
    "COMPLEX WORD COUNT":[],
    "WORD COUNT":[],
    "SYLLABLE PER WORD":[],
    "PERSONAL PRONOUNS":[],
    "AVG WORD LENGTH":[],
}

index = 0
url_id_list = list(df["URL_ID"])
for url in df["URL"]:
	url_id = url_id_list[index]
	index += 1
	source = requests.get(url, headers=headers).text
	data = BeautifulSoup(source, 'lxml')
	title = data.find("h1",class_="entry-title")
	desc = data.find("div",class_="td-post-content")
	if title is not None:
		title = title.text
	else:
		title = ""
	if desc is not None:
		desc = desc.text
	else:
		desc = ""
	all_text = title+" "+desc
	content = clean_text(all_text)
	newc = newContent(content)
	if len(newc) == 0:
		continue
	pword = open("/content/master/positive-words.txt","r").read().split("\n")
	nword = open("/content/master/negative-words.txt","r", encoding='utf-8').read().split("\n")	

	positive_score = 0
	negative_score = 0

	for word in newc:
	  if word.lower() in pword:
	    positive_score += 1
	  if word.lower() in nword:
	    negative_score += 1

	polarity_score = (positive_score - negative_score)/ ((positive_score + negative_score) + 0.000001)
	subjectivity_score = (positive_score + negative_score)/ (len(newc) + 0.000001)
	sentences= nltk.sent_tokenize(" ".join(all_text))

	complex_words = 0
	for word in newc:
	  if count_syllables(word) > 2:
	    complex_words += 1

	percent_complex_Word = (complex_words*100)/len(newc)

	syllables_per_word = complex_words/len(newc)
	average_sentence_length = len(newc)/len(sentences)
	fog_index = 0.4 * (average_sentence_length + complex_words)
	pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
	personal_pronouns = len(pronounRegex.findall(all_text))
	avg_word_len = (len(content)-len(newc))/len(newc)
	
	sents = all_text.split('.')
	avg_sent_len = sum(len(x.split()) for x in sents) / len(sents)

	output["URL_ID"].append(url_id)
	output["URL"].append(url)
	output["POSITIVE SCORE"].append(positive_score)
	output["NEGATIVE SCORE"].append(-negative_score)
	output["POLARITY SCORE"].append(polarity_score)
	output["SUBJECTIVITY SCORE"].append(subjectivity_score)
	output["AVG SENTENCE LENGTH"].append(avg_sent_len)
	output["PERCENTAGE OF COMPLEX WORDS"].append(percent_complex_Word)
	output["FOG INDEX"].append(fog_index)
	output["AVG NUMBER OF WORDS PER SENTENCE"].append(average_sentence_length)
	output["COMPLEX WORD COUNT"].append(complex_words)
	output["WORD COUNT"].append(len(newc))
	output["SYLLABLE PER WORD"].append(syllables_per_word)
	output["PERSONAL PRONOUNS"].append(personal_pronouns)
	output["AVG WORD LENGTH"].append(avg_word_len)
	
output_df = pd.DataFrame(output)
output_df.to_excel("Output_Data_Structure.xlsx")
print("output saved!")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


output saved!


In [168]:
odf = pd.DataFrame(output)

In [169]:
odf.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,88,-33,0.454545,0.109403,23.064103,34.900542,203.555556,122.888889,386,1106,0.349005,1,9.616637
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,84,-38,0.377049,0.162667,20.028169,26.266667,98.800000,50.000000,197,750,0.262667,7,9.854667
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,76,-41,0.299145,0.120247,20.117647,37.307297,274.933333,324.333333,363,973,0.373073,3,10.053443
3,40.0,https://insights.blackcoffer.com/will-machine-...,84,-32,0.448276,0.137441,20.012048,25.473934,103.768421,44.421053,215,844,0.254739,17,10.219194
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,67,-30,0.381443,0.098079,21.414634,28.210313,134.870588,58.176471,279,989,0.282103,16,9.555106


In [170]:
odf.to_excel("Output_Data_Structure.xlsx")